# Datacube extension testing in the Resource Catalogue

## Introduction

This notebook is used for testing datacube support in the Resource Catalogue. \
Eventually, it will be integrated into the EOEPCA notebook 04 Resource Catalogue. \

## Description

The notebook is divided into several sections to make it easier to understand.



## Section 1: Imports and EOEPCA connection setup
This section includes importing everything from the 04 notebook. \
It also includes the initial code blocks as these are used to setup the domain variables. \
Some information related to csw and the domain is printed as a check.

In [1]:
from owslib.csw import CatalogueServiceWeb
from owslib.ogcapi.records import Records
from owslib.fes import And, Or, PropertyIsEqualTo, PropertyIsGreaterThanOrEqualTo, PropertyIsLessThanOrEqualTo, PropertyIsLike, BBox, SortBy, SortProperty
from geolinks import sniff_link


In [79]:
# We use the develop domain as it has Datacube examples 

#base_domain = "demo.eoepca.org"
base_domain = "develop.eoepca.org"
#base_domain = "185.52.193.87.nip.io"
#workspace_prefix = "demo-user"
workspace_prefix = "user"

In [80]:
system_catalogue_endpoint = f'https://resource-catalogue.{base_domain}/csw'
print(system_catalogue_endpoint)

csw = CatalogueServiceWeb(system_catalogue_endpoint, timeout=30)

print(csw.identification.type)
print(csw.version)
print([op.name for op in csw.operations])

https://resource-catalogue.develop.eoepca.org/csw
CSW
2.0.2
['GetCapabilities', 'DescribeRecord', 'GetDomain', 'GetRecords', 'GetRecordById', 'GetRepositoryItem', 'Transaction', 'Harvest']


In [81]:
csw.get_operation_by_name('GetRecords').constraints

[Constraint: AdditionalQueryables - ['apiso:AccessConstraints', 'apiso:Bands', 'apiso:Classification', 'apiso:CloudCover', 'apiso:ConditionApplyingToAccessAndUse', 'apiso:Contributor', 'apiso:Creator', 'apiso:Degree', 'apiso:Instrument', 'apiso:Lineage', 'apiso:OtherConstraints', 'apiso:Platform', 'apiso:Publisher', 'apiso:Relation', 'apiso:ResponsiblePartyRole', 'apiso:SensorType', 'apiso:SpecificationDate', 'apiso:SpecificationDateType', 'apiso:SpecificationTitle'],
 Constraint: SupportedDublinCoreQueryables - ['csw:AnyText', 'dc:contributor', 'dc:creator', 'dc:date', 'dc:format', 'dc:identifier', 'dc:language', 'dc:publisher', 'dc:relation', 'dc:rights', 'dc:source', 'dc:subject', 'dc:title', 'dc:type', 'dct:abstract', 'dct:alternative', 'dct:modified', 'dct:spatial', 'ows:BoundingBox'],
 Constraint: SupportedISOQueryables - ['apiso:Abstract', 'apiso:AlternateTitle', 'apiso:AnyText', 'apiso:BoundingBox', 'apiso:CRS', 'apiso:CouplingType', 'apiso:CreationDate', 'apiso:Denominator', '

## Section 2: Discovering Datacubes

The second section involves some code to find and identify the presence of datacubes.\
A couple of datacube examples should be present on the develop cluster.\
These will later be accessed using different methods. 

Currently there are 3 examples available on the EOEPCA develop cluster:

[PIC 1 Month 9km](https://resource-catalogue.develop.eoepca.org/collections/metadata:main/items/PIC_1-month_9km)

[BICEP_NCEO_PIC_ESA-OC-L3S-OC](https://resource-catalogue.develop.eoepca.org/collections/metadata:main/items/BICEP_NCEO_PIC_ESA-OC-L3S-OC_PRODUCTS-MERGED-1M_MONTHLY_9km_GEO_PML_OCx_QAA-199711-fv4.2.nc)

[ESACCI-LC-L4-LCCS-Map-300m-P1Y-2009](https://resource-catalogue.develop.eoepca.org/collections/metadata:main/items/ESACCI-LC-L4-LCCS-Map-300m-P1Y-2009-v2.0.7cds)

In [82]:
bbox_query = BBox([-180, -90, 180, 90])

#begin = PropertyIsGreaterThanOrEqualTo(propertyname='apiso:TempExtent_begin', literal='2009-01-10 00:00')

#end = PropertyIsLessThanOrEqualTo(propertyname='apiso:TempExtent_end', literal='2009-12-31 00:00')

title = PropertyIsEqualTo(propertyname='apiso:Title', literal='ESA CCI Land Cover Map for 2009')


In [83]:
filter_list=[
    And(
        [
           bbox_query,
           title,
        ]
    )
]



In [84]:
item_identifier = 'ESACCI-LC-L4-LCCS-Map-300m-P1Y-2009-v2.0.7cds'

csw.getrecordbyid([item_identifier])

print(dir(record))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'abstract', 'accessrights', 'alternative', 'bbox', 'bbox_wgs84', 'contributor', 'coverage', 'created', 'creator', 'date', 'format', 'identifier', 'identifiers', 'ispartof', 'issued', 'language', 'license', 'modified', 'publisher', 'rdf', 'references', 'relation', 'rights', 'rightsholder', 'source', 'spatial', 'subjects', 'temporal', 'title', 'type', 'uris', 'xml']


In [85]:
record = csw.records[item_identifier]

print("Title:", record.title)
print("Abstract:", record.abstract)
print("Date:", record.date)
print("Type:", record.type)
print("Coverate:", record.coverage)
print("Format", record.format)
print("License:", record.license)
print("Source:", record.source)
print("Spatial:", record.spatial)


bbox = record.bbox
print("Bounding Box Dimensions:")
print("Min X:", bbox.minx)
print("Max X:", bbox.maxx)
print("Min Y:", bbox.miny)
print("Max Y:", bbox.maxy)

Title: ESA CCI Land Cover Map for 2009
Abstract: None
Date: None
Type: item
Coverate: None
Format None
License: None
Source: None
Spatial: None
Bounding Box Dimensions:
Min X: -180.0
Max X: 180.0
Min Y: -90.0
Max Y: 90.0
